<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Visualizing Survey Property Maps</b> <br>
Contact author(s): Eli Rykoff <br>
Last verified to run: 2022-07-24 <br>
LSST Science Piplines version: Weekly 2022_22 <br>
Container Size: large <br>
Targeted learning level: intermediate <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** Demonstrate tools to visualize survey property maps.

**Skills:** Load and visualize survey property maps using healsparse and skyproj.

**LSST Data Products:** Survey property maps.

**Packages:** healsparse, skyproj, lsst.daf.butler

**Credit:**
This notebook was originally developed by Eli Rykoff with editing from Alex Drlica-Wagner. Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook will teach the user how to load and visualize survey property maps generated by the Rubin Science Pipelines. Data products are accessed through the Butler, and the user is expected to be familiar with the content of the introductory Butler tutorial. It introduces two new packages, `healsparse` and `skyproj`, for working with survey property maps.

### 1.1 Package Imports

Import general python packages and the Butler from the science pipelines. Import two additional packages for working with the survey property maps. The `healsparse` packages provides utilities for reading and manipulating sparse healpix mpas, and documentation can be found [here](https://healsparse.readthedocs.io/en/latest/). The `skyproj` package provides utilities for visualizing both sparse and dense HEALPix maps. Documentation on skyproj can be found [here](https://skyproj.readthedocs.io/en/latest/).

In [ ]:
# general python packages
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# custom python packages for working with sparse healpix maps
import healsparse as hsp
import skyproj

# LSST package for Butler queries
from lsst.daf.butler import Butler

# This will allow us to have interactive plots
%matplotlib widget

# Note: This will trigger a warning from CFITSIO in w_2022_22.
# This warning can be safely ignored and will be corrected in the future.

## 2. Access Survey Property Maps

Survey property maps are created as part of the LSST Science Pipelines. They take the form of sparse HEALPix maps, where the survey property at each spatial pixel is identified by a pixel number/pixel value pair. You can read more about the healsparse format [here](https://healsparse.readthedocs.io/en/latest/). We start by creating an instance of the Butler and using it to access these maps for DP0.

In [ ]:
# We start by creating an instance of the Butler pointing to DP0.2
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collections)

In [ ]:
# Let's determine which property maps are available for the survey
for dtype in sorted(butler.registry.queryDatasetTypes(expression="*consolidated_map*")):
    print(dtype.name)

Each of these products represents a healsparse map containing the value of an individual survey property. The meaning of these types is:

* `deepCoadd_dcr_ddec_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) in declination direction
* `deepCoadd_dcr_dra_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) in right ascension direction
* `deepCoadd_dcr_e1_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) on psf e1
* `deepCoadd_dcr_e2_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) on psf e2
* `deepCoadd_exposure_time_consolidated_map_sum`: Total exposure time (seconds)
* `deepCoadd_psf_e1_consolidated_map_weighted_mean`: Weighted mean of psf e1 of images input to coadd
* `deepCoadd_psf_e2_consolidated_map_weighted_mean`: Weighted mean of psf e2 of images input to coadd
* `deepCoadd_psf_maglim_consolidated_map_weighted_mean`: PSF Flux 5-sigma magnitude limit (AB)
* `deepCoadd_psf_size_consolidated_map_weighted_mean`: Weighted mean of psf size of images input to coadd (pixels)
* `deepCoadd_sky_background_consolidated_map_weighted_mean`: Weighted mean of sky background of images input to coadd (ADU)
* `deepCoadd_sky_noise_consolidated_map_weighted_mean`: Weighted mean of sky noise of images input to coadd (ADU)





Note that the DCR maps are proportionality maps; that is, the expected effect will be proportional to the value in the map with an arbitrary/empirically derived constant of proportionality.

In [ ]:
# In order to read a map, we need to specify the map name and a band.
hspmap = butler.get('deepCoadd_psf_maglim_consolidated_map_weighted_mean', band='i')

## 3. Manipulating Survey Property Maps

The survey property maps are provided as healsparse objects. Much more detail on working with healsparse can be found in the documentation [here](https://healsparse.readthedocs.io/en/latest/). We provide a few very brief examples here.

To conserve memory, HealSparse uses a dual-map approach, where a low-resolution full-sky “coverage map” is combined with a high resolution map containing the pixel data where it is available. It is easy to find teh resolution of these maps.

In [ ]:
print(hspmap)

Each pixel of the healsparse map corresponds to a small region of the sky. The value of the map corresponds to the value of the survey property at that location. To access the survey property value at a specific location or set of locations.

In [ ]:
# Query for the map value at a specific location
print(hspmap.get_values_pos(60, -37))

# Query for the map value at an array of locations
ra = np.linspace(59.5,60.5)
dec = np.linspace(-37.5,-36.5)
print(hspmap.get_values_pos(ra, dec))

If you ask for the value of the map outside of the region where it is defined, you will get a sentinel value.

In [ ]:
print(hspmap.get_values_pos(180, 0))

## 4. Visualizing Survey Property Maps

Now that we know how to access the values of the healsparse map, we can put together our own simple visualization by creating a grid of RA, Dec. values and asking for the map values. We can then plot these values with matplotlib. Note that if you pan or zoom, the map does not update in resolution or coverage area.

In [ ]:
# Simple map visualization
ra = np.linspace(59.5,60.5, 50)
dec = np.linspace(-37.5,-36.5, 50)
x,y = np.meshgrid(ra,dec)
values = hspmap.get_values_pos(x,y)

fig, ax = plt.subplots(num=1, figsize=(10, 7))
plt.pcolormesh(x, y, values)
plt.xlabel("Declination (deg)")
plt.ylabel("Right Ascension (deg)")
plt.colorbar(label="PSF Maglim (i-band)")
plt.show()

The `skyproj` package provides much more advanced plotting capabilities (full documentation [here](https://skyproj.readthedocs.io/en/latest/)). Here we will demonstrate some basic use-cases for dealing with survey property maps. Note that we create a new figure number (`num = 2`) to avoid overplotting on our previous figure.

In [ ]:
# The map will be interactive in notebook mode
fig, ax = plt.subplots(num=2, figsize=(8, 5))
# This creates a visualization using the McBryde-Thomas Flat Polar Quartic, centered at 65 deg longitude, 
#  appropriate for the DP0.2 footprint
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

Notice that the edges of the survey are pulling the color scale and making it hard to see variations in the main survey area. We can create another visualization with a new colormap range to emphasize these smaller veriations.


In [ ]:
fig, ax = plt.subplots(num=3, figsize=(8, 5))

sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=26.0, vmax=26.3)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

The above maps are interactive thanks to our use of `%matplotlib widget` at the beginning of the notebook.  You can use the matplotlib "Zoom to rectangle" tool to draw a box and zoom in.  As you move the mouse around the map, the bottom label will show the longitude, latitude, and map value.  When you zoom in the default is that the colorbar will autoscale according to the data in the box.  If you want to turn this off you can specify `sp.set_autorescale(False)`

You can also zoom on a specific longitude/latitude range, as shown below.

In [ ]:
# And we can zoom in initially as well.  I've changed the vmin/vmax to make it look better.
fig, ax = plt.subplots(num=4, figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=26.0, vmax=26.3, lon_range=[55, 60], lat_range=[-40, -35])
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()